# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 11:27AM,247259,10,SOIVA0000533,Ivaoes Animal Health,Released
1,Sep 26 2022 11:27AM,247259,10,SOIVA0000532,Ivaoes Animal Health,Released
2,Sep 26 2022 11:23AM,247254,10,8563971,Eywa Pharma Inc.,Released
3,Sep 26 2022 11:23AM,247254,10,8564546,Eywa Pharma Inc.,Released
4,Sep 26 2022 11:23AM,247254,10,8565106,Eywa Pharma Inc.,Released
5,Sep 26 2022 11:23AM,247254,10,8565661,Eywa Pharma Inc.,Released
6,Sep 26 2022 11:23AM,247254,10,8574767,Eywa Pharma Inc.,Released
7,Sep 26 2022 11:23AM,247254,10,8574804,Eywa Pharma Inc.,Released
8,Sep 26 2022 11:23AM,247254,10,8574816,Eywa Pharma Inc.,Released
9,Sep 26 2022 11:19AM,247258,19,60018612,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,247254,Released,7
21,247256,Released,2
22,247257,Released,54
23,247258,Released,1
24,247259,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247254,NaN,7.0
247256,NaN,2.0
247257,NaN,54.0
247258,NaN,1.0
247259,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247197,1.0,0.0
247199,1.0,0.0
247219,0.0,15.0
247220,0.0,3.0
247223,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247197,1,0
247199,1,0
247219,0,15
247220,0,3
247223,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247197,1,0
1,247199,1,0
2,247219,0,15
3,247220,0,3
4,247223,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247197,1,
1,247199,1,
2,247219,,15
3,247220,,3
4,247223,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health
2,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.
9,Sep 26 2022 11:19AM,247258,19,"GUSA Granules USA, Inc."
10,Sep 26 2022 11:17AM,247257,10,ISDIN Corporation
64,Sep 26 2022 11:08AM,247256,10,"Methapharm, Inc."
66,Sep 26 2022 10:46AM,247246,21,"NBTY Global, Inc."
67,Sep 26 2022 10:41AM,247253,15,"Carwin Pharmaceutical Associates, LLC"
73,Sep 26 2022 10:40AM,247249,15,"Brookfield Pharmaceuticals, LLC"
97,Sep 26 2022 10:39AM,247248,15,"Alliance Pharma, Inc."
105,Sep 26 2022 10:29AM,247252,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,,2
1,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,,7
2,Sep 26 2022 11:19AM,247258,19,"GUSA Granules USA, Inc.",,1
3,Sep 26 2022 11:17AM,247257,10,ISDIN Corporation,,54
4,Sep 26 2022 11:08AM,247256,10,"Methapharm, Inc.",,2
5,Sep 26 2022 10:46AM,247246,21,"NBTY Global, Inc.",1,
6,Sep 26 2022 10:41AM,247253,15,"Carwin Pharmaceutical Associates, LLC",,6
7,Sep 26 2022 10:40AM,247249,15,"Brookfield Pharmaceuticals, LLC",,24
8,Sep 26 2022 10:39AM,247248,15,"Alliance Pharma, Inc.",2,6
9,Sep 26 2022 10:29AM,247252,20,Alumier Labs Inc.,5,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,2,
1,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,7,
2,Sep 26 2022 11:19AM,247258,19,"GUSA Granules USA, Inc.",1,
3,Sep 26 2022 11:17AM,247257,10,ISDIN Corporation,54,
4,Sep 26 2022 11:08AM,247256,10,"Methapharm, Inc.",2,
5,Sep 26 2022 10:46AM,247246,21,"NBTY Global, Inc.",,1
6,Sep 26 2022 10:41AM,247253,15,"Carwin Pharmaceutical Associates, LLC",6,
7,Sep 26 2022 10:40AM,247249,15,"Brookfield Pharmaceuticals, LLC",24,
8,Sep 26 2022 10:39AM,247248,15,"Alliance Pharma, Inc.",6,2
9,Sep 26 2022 10:29AM,247252,20,Alumier Labs Inc.,,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,2,
1,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,7,
2,Sep 26 2022 11:19AM,247258,19,"GUSA Granules USA, Inc.",1,
3,Sep 26 2022 11:17AM,247257,10,ISDIN Corporation,54,
4,Sep 26 2022 11:08AM,247256,10,"Methapharm, Inc.",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,2.0,NaN
1,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,7.0,NaN
2,Sep 26 2022 11:19AM,247258,19,"GUSA Granules USA, Inc.",1.0,NaN
3,Sep 26 2022 11:17AM,247257,10,ISDIN Corporation,54.0,NaN
4,Sep 26 2022 11:08AM,247256,10,"Methapharm, Inc.",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,2.0,0.0
1,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,7.0,0.0
2,Sep 26 2022 11:19AM,247258,19,"GUSA Granules USA, Inc.",1.0,0.0
3,Sep 26 2022 11:17AM,247257,10,ISDIN Corporation,54.0,0.0
4,Sep 26 2022 11:08AM,247256,10,"Methapharm, Inc.",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3461336,195.0,6.0
15,741750,36.0,2.0
16,247238,1.0,0.0
19,247258,1.0,0.0
20,247252,0.0,5.0
21,741642,0.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3461336,195.0,6.0
1,15,741750,36.0,2.0
2,16,247238,1.0,0.0
3,19,247258,1.0,0.0
4,20,247252,0.0,5.0
5,21,741642,0.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,195.0,6.0
1,15,36.0,2.0
2,16,1.0,0.0
3,19,1.0,0.0
4,20,0.0,5.0
5,21,0.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,195.0
1,15,Released,36.0
2,16,Released,1.0
3,19,Released,1.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,6.0,2.0,0.0,0.0,5.0,3.0
Released,195.0,36.0,1.0,1.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,6.0,2.0,0.0,0.0,5.0,3.0
1,Released,195.0,36.0,1.0,1.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,6.0,2.0,0.0,0.0,5.0,3.0
1,Released,195.0,36.0,1.0,1.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()